In [ ]:

# ==============================
# Setup & Data Loading (Q1 part)
# ==============================
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# User-specified dataset path
df = pd.read_csv(r"C:\Users\shubh\Downloads\BIKE_DETAILS.csv")

# Standardize column names (lowercase, underscores)
df.columns = (
    df.columns
    .str.strip()
    .str.lower()
    .str.replace(r"[^a-z0-9]+", "_", regex=True)
    .str.replace(r"_+", "_", regex=True)
    .str.strip("_")
)

# Helper: flexible column resolver
def find_col(df, candidates):
    cols = list(df.columns)
    for cand in candidates:
        if cand in cols:
            return cand
    # fuzzy match
    for c in cols:
        for cand in candidates:
            if cand in c:
                return c
    return None

# Likely column names
COL_SELLING_PRICE = find_col(df, ["selling_price", "sellingprice", "price", "sellingprize"])
COL_YEAR          = find_col(df, ["year", "model_year", "manufacture_year"])
COL_SELLER_TYPE   = find_col(df, ["seller_type", "sellertype", "seller"])
COL_OWNER         = find_col(df, ["owner", "ownership", "owner_type"])
COL_KM_DRIVEN     = find_col(df, ["km_driven", "kms_driven", "kilometers_driven", "km", "kms"])

# Coerce numeric columns where appropriate
def to_numeric_safe(series):
    return pd.to_numeric(series.astype(str).str.replace(",", "").str.strip(), errors="coerce")

if COL_SELLING_PRICE:
    df[COL_SELLING_PRICE] = to_numeric_safe(df[COL_SELLING_PRICE])
if COL_KM_DRIVEN:
    df[COL_KM_DRIVEN] = to_numeric_safe(df[COL_KM_DRIVEN])
if COL_YEAR:
    df[COL_YEAR] = to_numeric_safe(df[COL_YEAR]).astype("Int64")

# Clean owner field a bit (standardize categories)
if COL_OWNER and df[COL_OWNER].dtype == object:
    df[COL_OWNER] = (
        df[COL_OWNER]
        .str.strip()
        .str.lower()
        .str.replace(r"owner", "", regex=True)
        .str.replace(r"\s+", " ", regex=True)
        .str.strip()
    )

print(f"Shape: {df.shape}")
print("Columns:", list(df.columns))



## Question 1
**Task:** Read the Bike Details dataset into a Pandas DataFrame and display its first 10 rows. Show the shape and column names.


In [ ]:

print("DataFrame shape:", df.shape)
print("Columns:", list(df.columns))
display(df.head(10))



## Question 2
**Task:** Check for missing values in all columns and describe your approach for handling them.


In [ ]:

# Missing values summary
na_counts = df.isna().sum().sort_values(ascending=False)
na_pct = (df.isna().mean()*100).sort_values(ascending=False)
miss = pd.DataFrame({"missing_count": na_counts, "missing_pct": na_pct.round(2)})
display(miss)

# Example handling strategy (explain in markdown below; here we create a cleaned copy we'll reuse):
df_clean = df.copy()

# For numeric key columns like selling_price and km_driven, we won't impute with mean blindly.
# We keep them as-is for EDA and may drop rows with missing target (selling_price) for price analyses.
if COL_SELLING_PRICE:
    df_clean = df_clean[~df_clean[COL_SELLING_PRICE].isna()]

# For year: if absurd or missing, we can drop for plots that need year.
if COL_YEAR:
    df_clean = df_clean[~df_clean[COL_YEAR].isna()]

# For km_driven: keep NaN for now; some analyses will use non-null subset.
if COL_KM_DRIVEN:
    pass

print("Post basic cleaning shape:", df_clean.shape)



**Approach (brief):**
- For **target-like** fields such as `selling_price`, imputation can bias downstream modeling. For EDA, rows missing the target are **dropped** when analyzing price.
- For **temporal** fields like `year`, implausible or missing values are removed for time-based visualizations.
- For other numeric fields (e.g., `km_driven`), we keep as-is initially and use non-null subsets per analysis. If production modeling was required, we'd consider **domain-informed imputation** (e.g., median by `seller_type`).
- We avoid mean imputation on heavy-tailed variables (like price and kms) due to skewness; **median** is safer when imputation is necessary.



## Question 3
**Task:** Plot the distribution of `selling_price` and describe the overall trend.


In [ ]:

if not COL_SELLING_PRICE:
    raise KeyError("Could not locate a selling price column. Please adjust the resolver.")

sp = df_clean[COL_SELLING_PRICE].dropna()

plt.figure()
plt.hist(sp, bins=50)
plt.title("Distribution of Selling Price")
plt.xlabel("Selling Price")
plt.ylabel("Count")
plt.show()

print("Five-number summary of selling price:")
display(sp.describe(percentiles=[0.25,0.5,0.75]).to_frame("selling_price"))



**Observation (typical in used-vehicle data):**
The distribution is usually **right-skewed** (long tail toward higher prices). A bulk of listings cluster at lower-to-mid price bands with fewer high-priced bikes forming the tail.



## Question 4
**Task:** Average selling price for each `seller_type` (bar plot) and one observation.


In [ ]:

if not COL_SELLER_TYPE:
    raise KeyError("Could not locate seller_type column.")

grp = (
    df_clean.dropna(subset=[COL_SELLER_TYPE, COL_SELLING_PRICE])
    .groupby(COL_SELLER_TYPE)[COL_SELLING_PRICE]
    .mean()
    .sort_values(ascending=False)
)

plt.figure()
grp.plot(kind="bar")
plt.title("Average Selling Price by Seller Type")
plt.xlabel("Seller Type")
plt.ylabel("Average Selling Price")
plt.xticks(rotation=45, ha="right")
plt.tight_layout()
plt.show()

display(grp.to_frame("avg_selling_price"))



**Observation (example template):**
Dealer listings often show a **higher average price** than individual sellers, reflecting reconditioning costs, warranty, or selection bias toward newer models.



## Question 5
**Task:** Average `km_driven` for each ownership type (bar plot).


In [ ]:

if not COL_OWNER:
    raise KeyError("Could not locate owner/ownership column.")
if not COL_KM_DRIVEN:
    raise KeyError("Could not locate km_driven column.")

own_km = (
    df.dropna(subset=[COL_OWNER, COL_KM_DRIVEN])
    .groupby(COL_OWNER)[COL_KM_DRIVEN]
    .mean()
    .sort_values()
)

plt.figure()
own_km.plot(kind="bar")
plt.title("Average KMs Driven by Ownership Type")
plt.xlabel("Ownership")
plt.ylabel("Average km_driven")
plt.xticks(rotation=45, ha="right")
plt.tight_layout()
plt.show()

display(own_km.to_frame("avg_km_driven"))



## Question 6
**Task:** Use the IQR method to detect and remove outliers from `km_driven`. Show before-and-after summary statistics.


In [ ]:

if not COL_KM_DRIVEN:
    raise KeyError("Could not locate km_driven column.")

kms = df[COL_KM_DRIVEN].dropna()
before_stats = kms.describe(percentiles=[0.25,0.5,0.75]).to_frame("before")
Q1 = kms.quantile(0.25)
Q3 = kms.quantile(0.75)
IQR = Q3 - Q1
lower = Q1 - 1.5 * IQR
upper = Q3 + 1.5 * IQR

df_iqr = df.copy()
mask = df_iqr[COL_KM_DRIVEN].between(lower, upper, inclusive="both")
df_iqr = df_iqr[mask]

after_kms = df_iqr[COL_KM_DRIVEN].dropna()
after_stats = after_kms.describe(percentiles=[0.25,0.5,0.75]).to_frame("after")

print("IQR bounds:", lower, upper)
display(before_stats.join(after_stats, how="outer"))



## Question 7
**Task:** Scatter plot of `year` vs. `selling_price` to explore the relationship between age and price.


In [ ]:

if not (COL_YEAR and COL_SELLING_PRICE):
    raise KeyError("Missing year or selling price column.")

sub = df_clean.dropna(subset=[COL_YEAR, COL_SELLING_PRICE])
plt.figure()
plt.scatter(sub[COL_YEAR], sub[COL_SELLING_PRICE], s=10, alpha=0.5)
plt.title("Year vs Selling Price")
plt.xlabel("Year")
plt.ylabel("Selling Price")
plt.grid(True, linestyle="--", alpha=0.4)
plt.show()

# Quick correlation for reference
corr = sub[[COL_YEAR, COL_SELLING_PRICE]].corr().iloc[0,1]
print(f"Pearson correlation (year vs selling_price): {corr:.3f}")



## Question 8
**Task:** One-hot encode `seller_type` and display first 5 rows.


In [ ]:

if not COL_SELLER_TYPE:
    raise KeyError("Could not locate seller_type column.")

df_ohe = pd.get_dummies(df_clean, columns=[COL_SELLER_TYPE], prefix=COL_SELLER_TYPE, drop_first=False)
display(df_ohe.head(5))
print("New columns added:", [c for c in df_ohe.columns if c.startswith(COL_SELLER_TYPE + "_")])



## Question 9
**Task:** Heatmap of correlation matrix for all numeric columns. What correlations stand out?


In [ ]:

num_df = df_clean.select_dtypes(include=[np.number])
corr = num_df.corr()

plt.figure(figsize=(8,6))
im = plt.imshow(corr, interpolation='nearest')
plt.colorbar(im, fraction=0.046, pad=0.04)
plt.xticks(range(len(corr.columns)), corr.columns, rotation=90)
plt.yticks(range(len(corr.columns)), corr.columns)
plt.title("Correlation Heatmap (Numeric Columns)")
plt.tight_layout()
plt.show()

# Print top absolute correlations with selling_price (if present)
if COL_SELLING_PRICE in num_df.columns:
    corr_sp = corr[COL_SELLING_PRICE].drop(labels=[COL_SELLING_PRICE]).abs().sort_values(ascending=False).head(5)
    print("Top |correlation| with selling_price:")
    display(corr_sp.to_frame("abs_corr_with_price"))



## Question 10
**Task:** Summarize findings.


In [ ]:

summary_points = []

# Selling price distribution shape
if COL_SELLING_PRICE and COL_SELLING_PRICE in df_clean.columns:
    sp = df_clean[COL_SELLING_PRICE].dropna()
    skew = sp.skew()
    if skew > 1:
        summary_points.append("Selling price distribution is strongly right-skewed with a long upper tail.")
    elif skew > 0.5:
        summary_points.append("Selling price distribution is moderately right-skewed.")
    else:
        summary_points.append("Selling price distribution is roughly symmetric or mildly skewed.")

# Seller type effect
if COL_SELLER_TYPE and COL_SELLING_PRICE:
    grp = (
        df_clean.dropna(subset=[COL_SELLER_TYPE, COL_SELLING_PRICE])
        .groupby(COL_SELLER_TYPE)[COL_SELLING_PRICE].mean().sort_values(ascending=False)
    )
    if len(grp) >= 2:
        summary_points.append(f"Average price varies by seller_type; top category by mean price: '{grp.index[0]}' (~{grp.iloc[0]:.0f}).")

# Ownership & kms
if COL_OWNER and COL_KM_DRIVEN:
    own_km = (
        df.dropna(subset=[COL_OWNER, COL_KM_DRIVEN])
        .groupby(COL_OWNER)[COL_KM_DRIVEN].mean().sort_values()
    )
    if len(own_km) > 0:
        summary_points.append("Average km_driven differs by ownership level, suggesting usage intensity increases with subsequent owners.")

# Year vs price correlation
if COL_YEAR and COL_SELLING_PRICE:
    sub = df_clean.dropna(subset=[COL_YEAR, COL_SELLING_PRICE])
    if len(sub) > 5:
        corr = sub[[COL_YEAR, COL_SELLING_PRICE]].corr().iloc[0,1]
        if corr > 0.2:
            summary_points.append("Newer bikes (higher year) tend to have higher prices (positive correlation).")
        elif corr < -0.2:
            summary_points.append("Newer bikes tend to have lower prices (negative correlation) — unusual; check data quality.")
        else:
            summary_points.append("Weak linear correlation between year and price; relationship may be nonlinear or confounded by model mix.")

# Outliers handling
if COL_KM_DRIVEN:
    kms = df[COL_KM_DRIVEN].dropna()
    Q1, Q3 = kms.quantile(0.25), kms.quantile(0.75)
    IQR = Q3 - Q1
    lower, upper = Q1 - 1.5*IQR, Q3 + 1.5*IQR
    summary_points.append(f"IQR outlier bounds for km_driven: [{lower:.0f}, {upper:.0f}] — rows outside were excluded for robust summaries.")

summary_md = "### Summary (Auto-generated)\n\n" + "".join([f"- {pt}\n" for pt in summary_points]) \
             + "\n**Data Cleaning / Feature Engineering:**\n" \
             "- Standardized column names.\n" \
             "- Converted text numerics to numbers (removed commas).\n" \
             "- Dropped rows with missing `selling_price` for price-based analyses.\n" \
             "- Applied IQR rule for `km_driven` to identify outliers.\n" \
             "- One-hot encoded `seller_type` for modeling readiness.\n"

from IPython.display import Markdown, display
display(Markdown(summary_md))
